In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the path to your dataset
train_dir = r"C:\Users\arekn\OneDrive\Desktop\archive\train"
test_dir = r"C:\Users\arekn\OneDrive\Desktop\archive\test"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% of data will be used for validation
)

# Use the same generator for both training and validation, but specify 'subset'
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # 80% of data for training
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,  # Still using the training directory for validation data
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # 20% of data for validation
)

# Print class indices to see the mappings of folders (letters A-Y) to numerical labels
print("Class Indices:", train_generator.class_indices)


Found 21974 images belonging to 24 classes.
Found 5481 images belonging to 24 classes.
Class Indices: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'K': 9, 'L': 10, 'M': 11, 'N': 12, 'O': 13, 'P': 14, 'Q': 15, 'R': 16, 'S': 17, 'T': 18, 'U': 19, 'V': 20, 'W': 21, 'X': 22, 'Y': 23}


In [9]:
from keras.layers import Input

# Model Architecture
model = Sequential([
    Input(shape=(64, 64, 3)),  # Define the input layer explicitly
    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [10]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('sign_language_model.keras', save_best_only=True, monitor='val_loss')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# Train the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=[early_stopping, checkpoint, reduce_lr]
)

# Save the final model
model.save('sign_language_final_model.keras')


Epoch 1/50


C:\Users\arekn\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


687/687 ━━━━━━━━━━━━━━━━━━━━ 240s 343ms/step - accuracy: 0.0701 - loss: 3.3282 - val_accuracy: 0.1394 - val_loss: 2.8082 - learning_rate: 0.0010
Epoch 2/50
687/687 ━━━━━━━━━━━━━━━━━━━━ 149s 216ms/step - accuracy: 0.1310 - loss: 2.8124 - val_accuracy: 0.1783 - val_loss: 2.6352 - learning_rate: 0.0010
Epoch 3/50
687/687 ━━━━━━━━━━━━━━━━━━━━ 107s 156ms/step - accuracy: 0.1973 - loss: 2.4872 - val_accuracy: 0.2385 - val_loss: 2.7208 - learning_rate: 0.0010
Epoch 4/50
687/687 ━━━━━━━━━━━━━━━━━━━━ 108s 156ms/step - accuracy: 0.2538 - loss: 2.2616 - val_accuracy: 0.3204 - val_loss: 2.0327 - learning_rate: 0.0010
Epoch 5/50
687/687 ━━━━━━━━━━━━━━━━━━━━ 103s 149ms/step - accuracy: 0.3142 - loss: 2.0233 - val_accuracy: 0.4527 - val_loss: 1.6742 - learning_rate: 0.0010
Epoch 6/50
687/687 ━━━━━━━━━━━━━━━━━━━━ 103s 149ms/step - accuracy: 0.3590 - loss: 1.8572 - val_accuracy: 0.3060 - val_loss: 2.2100 - learning_rate: 0.0010
Epoch 7/50
687/687 ━━━━━━━━━━━━━━━━━━━━ 103s 149ms/step - accuracy: 0.4051 

In [11]:
# Convert the Keras model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model to a file
tflite_model_path = 'sign_language_model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print(f"Model successfully converted to TFLite and saved as {tflite_model_path}")

INFO:tensorflow:Assets written to: C:\Users\arekn\AppData\Local\Temp\tmpa7o520jc\assets


INFO:tensorflow:Assets written to: C:\Users\arekn\AppData\Local\Temp\tmpa7o520jc\assets


Saved artifact at 'C:\Users\arekn\AppData\Local\Temp\tmpa7o520jc'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name='keras_tensor_28')
Output Type:
  TensorSpec(shape=(None, 24), dtype=tf.float32, name=None)
Captures:
  2582020088912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2582020087568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2582020090640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2582020086416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2582020089296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2582020090448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2582020091408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2582020092560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2582020092944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2582020091024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  258

In [5]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the image you want to test
img_path = r"C:\Users\arekn\OneDrive\Desktop\archive\Test\N\480_N.jpg"
img = image.load_img(img_path, target_size=(64, 64))

# Preprocess the image to match the input format of the model
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize the image just like in training

# Load the saved model
model = tf.keras.models.load_model('sign_language_final_model.keras')

# Make a prediction
predictions = model.predict(img_array)

# Get the predicted class index
predicted_class_index = np.argmax(predictions[0])

# Print the predicted class
class_indices = train_generator.class_indices  # Use the same class indices as used during training
class_labels = {v: k for k, v in class_indices.items()}  # Invert the dictionary to map indices back to class labels
predicted_label = class_labels[predicted_class_index]

print(f"Predicted class: {predicted_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
Predicted class: N


In [10]:
import numpy as np
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing import image
import tensorflow as tf

# Load your test data (assuming you have test_generator or test data in a similar format)
# For example, using ImageDataGenerator for test data
test_datagen = image.ImageDataGenerator(rescale=1.0/255.0)
test_generator = test_datagen.flow_from_directory(
    'C:\\Users\\arekn\\OneDrive\\Desktop\\archive\\test',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Important to set shuffle=False for consistent results
)

# Load the saved .keras model
model = tf.keras.models.load_model('sign_language_final_model.keras')

# Predict the classes for the test data
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

# Get the ground truth labels from the test generator
y_true = test_generator.classes

# Get class labels mapping from the generator (mapping from indices to class names)
class_labels = list(test_generator.class_indices.keys())

# Generate the classification report
report = classification_report(y_true, y_pred_classes, target_names=class_labels)
print(report)


Found 7172 images belonging to 24 classes.
  1/225 ━━━━━━━━━━━━━━━━━━━━ 1:06 298ms/step

C:\Users\arekn\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


225/225 ━━━━━━━━━━━━━━━━━━━━ 36s 158ms/step
              precision    recall  f1-score   support

           A       1.00      1.00      1.00       331
           B       1.00      1.00      1.00       432
           C       1.00      1.00      1.00       310
           D       0.97      1.00      0.99       245
           E       1.00      0.96      0.98       498
           F       1.00      1.00      1.00       247
           G       1.00      0.98      0.99       348
           H       0.99      1.00      0.99       436
           I       1.00      1.00      1.00       288
           K       1.00      0.98      0.99       331
           L       1.00      1.00      1.00       209
           M       0.95      0.99      0.97       394
           N       0.99      0.95      0.97       291
           O       1.00      1.00      1.00       246
           P       1.00      1.00      1.00       347
           Q       1.00      1.00      1.00       164
           R       0.97      0.98    